## Genome Annotation Viewer

This notebook contains code for viewing genome annotations in the form of gff or genbank files.

Links 

* https://bokeh.pydata.org/en/latest/docs/user_guide.html
* https://realpython.com/python-data-visualization-bokeh/

In [18]:
import os, sys, io, random
import string
import numpy as np
import pandas as pd

from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO, SeqIO

from IPython.display import HTML

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Plot, LinearAxis, Grid, Range1d,CustomJS, Slider, HoverTool, NumeralTickFormatter, Arrow, NormalHead
from bokeh.models.glyphs import Text, Rect
from bokeh.layouts import gridplot, column
import panel as pn
import panel.widgets as pnw
pn.extension()

sys.path.insert(0,'../')
from pybioviz import viewers, utils

## plot gff

In [19]:
def plot_features(features, x_range=None, fontsize="8pt", plot_width=800, plot_height=150):
    """Bokeh sequence alignment view"""
    
    df = utils.features_to_dataframe(features)#, cds=True)
    df = df[df.type!='region']    
    df['length'] = df.end-df.start
    df['level'] = 1
    df['color'] = utils.random_colors(len(df)) #'green'
    df['x'] = df.start+df.length/2
    df['end_x'] = df.start+50
    df['y'] = [random.choice(range(1,9)) for i in range(len(df))]
    
    #print (df[:3])
    text = df.gene
    S = df.start.min()
    N = df.end.max()+10        
    x = list(df.start+df.length/2)
    h = 20

    source = ColumnDataSource(df)    
        
    viewlen=5000
    if x_range == None:
        x_range = (0,viewlen)
    else:
        #viewlen = int(x_range[1])-int(x_range[0])
        print (x_range,viewlen)    

    hover = HoverTool(
        tooltips=[            
            ("gene", "@gene"),     
            ("locus_tag", "@locus_tag"),
            ("protein_id", "@protein_id"), 
            ("length", "@length"),             
        ],        
    )  
    tools=[hover,"xpan, xwheel_zoom, save"]
    
    #sequence text view with ability to scroll along x axis
    p = figure(title=None, plot_width=plot_width, plot_height=plot_height, x_range=x_range,
                y_range=(0,10), tools=tools, min_border=0, toolbar_location='right')#, lod_factor=1)
    #display text only at certain zoom level
    if viewlen<30000:
        tags = Text(x="x", y="y", y_offset=-5, text="gene", text_align='center',text_color="black", 
                     text_font="monospace",text_font_size=fontsize, name="genetext")
        p.add_glyph(source, tags)
    rects = Rect(x="x", y="y", width="length", height=.4, fill_color="color", fill_alpha=0.4, name='rects')
    #add arrows
    arr = Arrow(source=source, x_start="start", x_end="end_x", y_start="y", y_end="y", 
                line_color="black", name='arrows', end=NormalHead(size=10))
    p.add_glyph(source, rects)
    
    p.add_layout(arr)
    
    p.grid.visible = False
    p.yaxis.visible = False
    p.xaxis.major_label_text_font_style = "bold"
    p.yaxis.minor_tick_line_width = 0
    p.yaxis.major_tick_line_width = 0
    p.toolbar.logo = None
    p.xaxis.formatter = NumeralTickFormatter(format="(0,0)")
    
    '''if preview == True:
        #entire sequence view (no text, with zoom)
        p = figure(title=None, plot_width=plot_width, plot_height=100, x_range=x_range, y_range=(-2,2), tools=tools, 
                        min_border=0, toolbar_location='below')
        rects = Rect(x="x", y="strand", width="length", height=.5, fill_color="colors", line_color='black', fill_alpha=0.6)
        p.add_glyph(source, rects)
        p.yaxis.visible = False
        p.grid.visible = False

        jscode="""    
        var start = cb_obj.value;    
        x_range.setv({"start": start, "end": start+l})   
        """
        callback = CustomJS(
            args=dict(x_range=p1.x_range,l=viewlen), code=jscode)
        slider = Slider (start=1, end=N, value=1, step=100)
        slider.js_on_change('value', callback)
        p = gridplot([[p],[slider],[p1]], toolbar_location='below')
    else:
        p = p1'''
    return p

feats = utils.gff_to_features('RD900MAF.gff')
#feats = get_features('Mbovis_AF212297.gff')
#print (feats)
p = plot_features(feats, x_range=(100,6000), plot_width=900)
pn.pane.Bokeh(p)


(100, 6000) 5000


Bokeh(Figure)

In [3]:
def preview(features, plot_width=800, plot_height=100):
    df = utils.features_to_dataframe(features)#, cds=True)
    df = df[df.type!='region']
    #df['gene'] = df.gene.apply(lambda x: x.locus_tag)
    df['length'] = df.end-df.start
    df['level'] = 1
    df['color'] = 'green'
    df['x'] = df.start+df.length/2
    #entire sequence view (no text, with zoom)
    p = figure(title=None, plot_width=plot_width, plot_height=100, x_range=x_range, y_range=(-2,2), tools=tools, 
                    min_border=0, toolbar_location='below')
    rects = Rect(x="x", y="strand", width="length", height=.4, fill_color="colors", line_color='black', fill_alpha=0.6)
    p.add_glyph(source, rects)
    p.yaxis.visible = False
    p.grid.visible = False    
    return p

In [14]:
def plot_sequence(sequence, fontsize="8pt", plot_width=900):
    #plot ref sequence    
    x = list(range(len(sequence)))
    colors = utils.get_sequence_colors([sequence])
    text = [i for i in sequence]  
    x_range = (0,len(sequence))
    source = ColumnDataSource(dict(x=x, text=text, colors=colors))
    p = figure(title=None, plot_width=plot_width, plot_height=40, x_range=x_range, y_range=(0,1), tools="", 
                    min_border=0)
    if len(sequence)<4000:
        rects = Rect(x="x", y=0,  width=1, height=2, fill_color="colors", line_color=None, fill_alpha=0.6)
        p.add_glyph(source, rects)    
    if len(sequence)<200:
        seqtext = Text(x="x", y=0, text="text", text_align='center',text_color="black", text_font="monospace",text_font_size=fontsize)
        p.add_glyph(source, seqtext)
    
    p.grid.visible = False
    p.yaxis.visible = False
    p.toolbar.logo = None
    return p

def get_subsequence(fasta_file, start, end):    
    from pyfaidx import Fasta
    genes = Fasta('Mbovis-AF212297.fa')
    return str(genes[0][start:end])

#rec = SeqIO.read('Mbovis-AF212297.fa','fasta')
#sequence = rec.seq[1200:1320]
sequence = get_subsequence('Mbovis-AF212297.fa',100,200)
print (sequence)
p = plot_sequence(sequence, plot_width=900)
pn.pane.Bokeh(p)

CTAATCTCAGCGCTCCGCTGACCCCTCAGCAAAGGGCTTGGCTCAATCTCGTCCAGCCATTGACCATCGTCGAGGGGTTTGCTCTGTTATCCGTGCCGAG


Bokeh(Figure)

## test app

In [19]:
def view_features(features=None):
    """gene feature viewer app"""
    
    #features = utils.gff_to_features('Mbovis_AF212297.gff')
    gff_input = pnw.TextInput(name='gff file',value='',width=200)
    ref_input = pnw.TextInput(name='ref sequence file',value='Mbovis-AF212297.fa',width=200)
    loc_input = pnw.TextInput(name='location',value='',width=180)
    gene_input = pnw.TextInput(name='find_gene',value='',width=180)
    zoomout_btn = pnw.Button(name='-',width=40, button_type='primary')
    zoomin_btn = pnw.Button(name='+',width=40, button_type='primary')
    #load_btn = pn.widgets.FileInput()
    slider = pnw.IntRangeSlider(start=0,end=1000000,step=10,value=(1,20000),width=900)
    feature_pane = pn.pane.Bokeh(height=100,margin=10)
    seq_pane = pn.pane.Bokeh(height=50,margin=10)
    found = None
    if features is None:
        features = utils.gff_to_features(gff_input.value)
    
    def load_file(event):
        nonlocal features
        features = utils.gff_to_features(gff_input.value)
        #feature_pane.object = plot_features(features,preview=False,x_range=xrange, plot_width=900)
        update(event)
        
    def find_gene(event):
        gene = gene_input.value         
        df = utils.features_to_dataframe(features).fillna('-')
        found = df[df.gene.str.contains(gene)].iloc[0]        
        loc = (found.start-500,found.end+500)
        slider.value = loc
        #feature_pane.object = view_features(features,preview=False,x_range=loc, plot_width=900)
        return
    
    def zoomout(event):
        
        return
    
    def update(event):    
        xrange = slider.value
        loc_input.value = str(xrange[0])+':'+str(xrange[1])
        #p1 = annot_pane.object = preview(features)
        feature_pane.object = plot_features(features,preview=False,x_range=xrange, plot_width=900)
        sequence = get_subsequence(ref_input.value, xrange[0], xrange[1])
        seq_pane.object = plot_sequence(sequence)
        return

    slider.param.watch(update,'value')
    slider.param.trigger('value')
    gene_input.param.watch(find_gene,'value')
    gff_input.param.watch(load_file,'value')
    zoomout_btn.param.watch(zoomout,'clicks')
    test_pane = pn.pane.Str(50,width=180,style={'margin': '4pt'})
    
    top=pn.Row(gff_input,ref_input,loc_input,gene_input,zoomout_btn,zoomin_btn)
    main = pn.Column(feature_pane, seq_pane, sizing_mode='stretch_width')
    app = pn.Column(top,slider,main)
    return app

features = utils.gff_to_features('Mbovis_AF212297.gff')
app = view_features(features)
app

(1, 20000) 5000


Column
    [0] Row
        [0] TextInput(name='gff file', width=200)
        [1] TextInput(name='ref sequence file', value='Mbovis-AF212297.fa', width=200)
        [2] TextInput(name='location', value='1:20000', width=180)
        [3] TextInput(name='find_gene', width=180)
        [4] Button(button_type='primary', name='-', width=40)
        [5] Button(button_type='primary', name='+', width=40)
    [1] IntRangeSlider(end=1000000, step=10, value=(1, 20000), width=900)
    [2] Column(sizing_mode='stretch_width')
        [0] Bokeh(Figure, height=100, margin=10)
        [1] Bokeh(Figure, height=50, margin=10)

In [6]:
gene='gyr'
df = utils.features_to_dataframe(features).fillna('-')
found = df[df.gene.str.contains(gene)]#.iloc[0] 
print (found)
loc = (found.start-100,found.end+100)
print (loc)

   type protein_id      locus_tag  gene db_xref product note translation  \
5  gene          -  BQ2027_MB0005  gyrB       -       -    -           -   
6  gene          -  BQ2027_MB0006  gyrA       -       -    -           -   

  pseudo  start   end  strand  length  
5      -   5122  7267       1       3  
6      -   7301  9818       1       3  
(5    5022
6    7201
Name: start, dtype: int64, 5    7367
6    9918
Name: end, dtype: int64)


In [ ]:
'RD900MAF.gff'